In [1]:
import pandas as pd
import numpy as np
import requests
import bs4 as bs
import urllib.request

## Extracting features of 2018 movies from Wikipedia

In [2]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2018"

In [3]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [4]:
tables = soup.find_all('table',class_='wikitable sortable')

In [5]:
len(tables)

4

In [6]:
type(tables[0])

bs4.element.Tag

In [7]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0] # avoided "ValueError: invalid literal for int() with base 10: '1"'

In [8]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

C:\Users\chand\AppData\Local\Temp\ipykernel_13752\1752420719.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)


In [9]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.
0,J A N U A R Y,3.0,Paranormal Activity: The Marked Ones,Paramount Pictures,Christopher Landon (director/screenplay); Andr...,[2],NaN
1,J A N U A R Y,3.0,Open Grave,Tribeca Films / 852 Films,"Gonzálo López-Gallego (director); Chris Borey,...",[3],NaN
2,J A N U A R Y,10.0,Cold Comes the Night,Sony Pictures Worldwide Acquisitions,"Tze Chun (director/screenplay); Oz Perkins, Ni...",[4],NaN
3,J A N U A R Y,10.0,Dumbbells,GoDigital,Christopher Livingston (director); Brian Drole...,[5],NaN
4,J A N U A R Y,10.0,The Legend of Hercules,Summit Entertainment / Millennium Films,"Renny Harlin (director); Sean Hood, Daniel Gia...",[6],NaN
...,...,...,...,...,...,...,...
226,D E C E M B E R,25.0,Into the Woods,Walt Disney Pictures,Rob Marshall (director); James Lapine (screenp...,NaN,[228]
227,D E C E M B E R,25.0,Selma,Paramount Pictures / Plan B Entertainment,Ava DuVernay (director/screenplay); Paul Webb ...,NaN,[224]
228,D E C E M B E R,25.0,Unbroken,Universal Pictures / Legendary Pictures,"Angelina Jolie (director); Joel Coen, Ethan Co...",NaN,[229]
229,D E C E M B E R,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df_2018 = df[['Title','Cast and crew']]

In [11]:
df_2018

,Title,Cast and crew
0,Paranormal Activity: The Marked Ones,Christopher Landon (director/screenplay); Andr...
1,Open Grave,"Gonzálo López-Gallego (director); Chris Borey,..."
2,Cold Comes the Night,"Tze Chun (director/screenplay); Oz Perkins, Ni..."
3,Dumbbells,Christopher Livingston (director); Brian Drole...
4,The Legend of Hercules,"Renny Harlin (director); Sean Hood, Daniel Gia..."
...,...,...
226,Into the Woods,Rob Marshall (director); James Lapine (screenp...
227,Selma,Ava DuVernay (director/screenplay); Paul Webb ...
228,Unbroken,"Angelina Jolie (director); Joel Coen, Ethan Co..."
229,NaN,NaN


In [12]:
!pip install tmdbv3api

In [13]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = '703a12952f3292e9958bc4d2d904a228'

In [14]:
from tmdbv3api import Movie
tmdb_movie = Movie() 
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    if not result:
      return np.NaN
    else:
      movie_id = result[0].id
      response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
      data_json = response.json()
      if data_json['genres']:
          genre_str = " " 
          for i in range(0,len(data_json['genres'])):
              genres.append(data_json['genres'][i]['name'])
          return genre_str.join(genres)
      else:
          return np.NaN

In [15]:
df_2018['genres'] = df_2018['Title'].map(lambda x: get_genre(str(x)))

C:\Users\chand\AppData\Local\Temp\ipykernel_13752\2453146623.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2014['genres'] = df_2014['Title'].map(lambda x: get_genre(str(x)))


In [16]:
df_2018

,Title,Cast and crew,genres
0,Paranormal Activity: The Marked Ones,Christopher Landon (director/screenplay); Andr...,Horror Thriller
1,Open Grave,"Gonzálo López-Gallego (director); Chris Borey,...",Horror Thriller Mystery
2,Cold Comes the Night,"Tze Chun (director/screenplay); Oz Perkins, Ni...",Crime Drama Thriller
3,Dumbbells,Christopher Livingston (director); Brian Drole...,Comedy
4,The Legend of Hercules,"Renny Harlin (director); Sean Hood, Daniel Gia...",Action Adventure
...,...,...,...
226,Into the Woods,Rob Marshall (director); James Lapine (screenp...,Fantasy Comedy
227,Selma,Ava DuVernay (director/screenplay); Paul Webb ...,History Drama
228,Unbroken,"Angelina Jolie (director); Joel Coen, Ethan Co...",Drama War
229,NaN,NaN,Drama


In [17]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [18]:
df_2018['director_name'] = df_2018['Cast and crew'].map(lambda x: get_director(str(x)))

C:\Users\chand\AppData\Local\Temp\ipykernel_13752\1036864481.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2014['director_name'] = df_2014['Cast and crew'].map(lambda x: get_director(str(x)))


In [19]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [20]:
df_2018['actor_1_name'] = df_2018['Cast and crew'].map(lambda x: get_actor1(str(x)))

C:\Users\chand\AppData\Local\Temp\ipykernel_13752\3177064408.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2014['actor_1_name'] = df_2014['Cast and crew'].map(lambda x: get_actor1(str(x)))


In [21]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [22]:
df_2018['actor_2_name'] = df_2018['Cast and crew'].map(lambda x: get_actor2(str(x)))

C:\Users\chand\AppData\Local\Temp\ipykernel_13752\4091272564.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2014['actor_2_name'] = df_2014['Cast and crew'].map(lambda x: get_actor2(str(x)))


In [23]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [24]:
df_2018['actor_3_name'] = df_2018['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [25]:
df_2018

,Title,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,Paranormal Activity: The Marked Ones,Christopher Landon (director/screenplay); Andr...,Horror Thriller,Christopher Landon,Andrew Jacobs,Jorge Diaz,Gabrielle Walsh
1,Open Grave,"Gonzálo López-Gallego (director); Chris Borey,...",Horror Thriller Mystery,Gonzálo López-Gallego,Sharlto Copley,Thomas Kretschmann,Joseph Morgan
2,Cold Comes the Night,"Tze Chun (director/screenplay); Oz Perkins, Ni...",Crime Drama Thriller,Tze Chun,Alice Eve,Logan Marshall-Green,Bryan Cranston
3,Dumbbells,Christopher Livingston (director); Brian Drole...,Comedy,Christopher Livingston,Brian Drolet,Hoyt Richards,Taylor Cole
4,The Legend of Hercules,"Renny Harlin (director); Sean Hood, Daniel Gia...",Action Adventure,Renny Harlin,Kellan Lutz,Gaia Weiss,Scott Adkins
...,...,...,...,...,...,...,...
226,Into the Woods,Rob Marshall (director); James Lapine (screenp...,Fantasy Comedy,Rob Marshall,Meryl Streep,Emily Blunt,James Corden
227,Selma,Ava DuVernay (director/screenplay); Paul Webb ...,History Drama,Ava DuVernay,David Oyelowo,Tom Wilkinson,Common
228,Unbroken,"Angelina Jolie (director); Joel Coen, Ethan Co...",Drama War,Angelina Jolie,Jack O'Connell,Domhnall Gleeson,Miyavi
229,NaN,NaN,Drama,nan,nan,NaN,NaN


In [26]:
df_2018 = df_2018.rename(columns={'Title':'movie_title'})

In [27]:
new_df18 = df_2018.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [28]:
new_df18

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Christopher Landon,Andrew Jacobs,Jorge Diaz,Gabrielle Walsh,Horror Thriller,Paranormal Activity: The Marked Ones
1,Gonzálo López-Gallego,Sharlto Copley,Thomas Kretschmann,Joseph Morgan,Horror Thriller Mystery,Open Grave
2,Tze Chun,Alice Eve,Logan Marshall-Green,Bryan Cranston,Crime Drama Thriller,Cold Comes the Night
3,Christopher Livingston,Brian Drolet,Hoyt Richards,Taylor Cole,Comedy,Dumbbells
4,Renny Harlin,Kellan Lutz,Gaia Weiss,Scott Adkins,Action Adventure,The Legend of Hercules
...,...,...,...,...,...,...
226,Rob Marshall,Meryl Streep,Emily Blunt,James Corden,Fantasy Comedy,Into the Woods
227,Ava DuVernay,David Oyelowo,Tom Wilkinson,Common,History Drama,Selma
228,Angelina Jolie,Jack O'Connell,Domhnall Gleeson,Miyavi,Drama War,Unbroken
229,nan,nan,NaN,NaN,Drama,NaN


In [29]:
new_df18['comb'] = new_df18['actor_1_name'] + ' ' + new_df18['actor_2_name'] + ' '+ new_df18['actor_3_name'] + ' '+ new_df18['director_name'] +' ' + new_df18['genres']

In [30]:
new_df18.isna().sum()

director_name     0
actor_1_name      0
actor_2_name      5
actor_3_name     12
genres            0
movie_title       2
comb             12
dtype: int64

In [31]:
new_df18 = new_df18.dropna(how='any')

In [32]:
new_df18.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64

In [33]:
new_df18['movie_title'] = new_df18['movie_title'].str.lower()

C:\Users\chand\AppData\Local\Temp\ipykernel_13752\696104642.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df14['movie_title'] = new_df14['movie_title'].str.lower()


In [34]:
new_df18

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Christopher Landon,Andrew Jacobs,Jorge Diaz,Gabrielle Walsh,Horror Thriller,paranormal activity: the marked ones,Andrew Jacobs Jorge Diaz Gabrielle Walsh Chris...
1,Gonzálo López-Gallego,Sharlto Copley,Thomas Kretschmann,Joseph Morgan,Horror Thriller Mystery,open grave,Sharlto Copley Thomas Kretschmann Joseph Morga...
2,Tze Chun,Alice Eve,Logan Marshall-Green,Bryan Cranston,Crime Drama Thriller,cold comes the night,Alice Eve Logan Marshall-Green Bryan Cranston ...
3,Christopher Livingston,Brian Drolet,Hoyt Richards,Taylor Cole,Comedy,dumbbells,Brian Drolet Hoyt Richards Taylor Cole Christo...
4,Renny Harlin,Kellan Lutz,Gaia Weiss,Scott Adkins,Action Adventure,the legend of hercules,Kellan Lutz Gaia Weiss Scott Adkins Renny Harl...
...,...,...,...,...,...,...,...
224,Rupert Wyatt,Mark Wahlberg,Brie Larson,Jessica Lange,Thriller Crime Drama,the gambler,Mark Wahlberg Brie Larson Jessica Lange Rupert...
225,"Seth Rogen, Evan Goldberg",James Franco,Seth Rogen,Lizzy Caplan,Action Comedy,the interview,James Franco Seth Rogen Lizzy Caplan Seth Roge...
226,Rob Marshall,Meryl Streep,Emily Blunt,James Corden,Fantasy Comedy,into the woods,Meryl Streep Emily Blunt James Corden Rob Mars...
227,Ava DuVernay,David Oyelowo,Tom Wilkinson,Common,History Drama,selma,David Oyelowo Tom Wilkinson Common Ava DuVerna...


In [35]:
old_df = pd.read_csv('final_data.csv')

In [36]:
old_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
5864,Greta Gerwig,Saoirse Ronan,Emma Watson,Florence Pugh,Drama Romance,little women,Saoirse Ronan Emma Watson Florence Pugh Greta ...
5865,Sam Mendes,George MacKay,Dean-Charles Chapman,Mark Strong,War Drama Action History,1917,George MacKay Dean-Charles Chapman Mark Strong...
5866,Destin Daniel Cretton,Michael B. Jordan,Jamie Foxx,Brie Larson,Drama Crime,just mercy,Michael B. Jordan Jamie Foxx Brie Larson Desti...
5867,Chinonye Chukwu,Alfre Woodard,Wendell Pierce,Aldis Hodge,Drama,clemency,Alfre Woodard Wendell Pierce Aldis Hodge Chino...


In [37]:
final_df = old_df.append(new_df18,ignore_index=True)

C:\Users\chand\AppData\Local\Temp\ipykernel_13752\914367843.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = old_df.append(new_df14,ignore_index=True)


In [38]:
final_df

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,Action Adventure Fantasy Sci-Fi,avatar,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,Action Adventure Fantasy,pirates of the caribbean: at world's end,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,Action Adventure Thriller,spectre,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,Action Thriller,the dark knight rises,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,Documentary,star wars: episode vii - the force awakens ...,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
6083,Rupert Wyatt,Mark Wahlberg,Brie Larson,Jessica Lange,Thriller Crime Drama,the gambler,Mark Wahlberg Brie Larson Jessica Lange Rupert...
6084,"Seth Rogen, Evan Goldberg",James Franco,Seth Rogen,Lizzy Caplan,Action Comedy,the interview,James Franco Seth Rogen Lizzy Caplan Seth Roge...
6085,Rob Marshall,Meryl Streep,Emily Blunt,James Corden,Fantasy Comedy,into the woods,Meryl Streep Emily Blunt James Corden Rob Mars...
6086,Ava DuVernay,David Oyelowo,Tom Wilkinson,Common,History Drama,selma,David Oyelowo Tom Wilkinson Common Ava DuVerna...


In [39]:
final_df.to_csv('main_data.csv',index=False)